## Q1. Chi Square Test

In [1]:
#TODO


import pandas as pd
import json
import numpy as np

business_path = "/Users/Andy/Desktop/yelp_academic_dataset_business.json"
business_doc=open(business_path, encoding="utf8").readlines()

busList = []
for bus in business_doc:
    busList.append(json.loads(bus))

dfBus = pd.DataFrame.from_records(busList)
dfBus = dfBus.dropna()
from pandas.io.json import json_normalize

In [2]:
open_df = dfBus.loc[:,'is_open']
estrella_df = dfBus.loc[:,'stars']

stars = estrella_df.values

bin_stars = [1 if (star > 2.5) else 0 for star in stars]

bin_stars = np.array(bin_stars) #1 for above 2.5 stars 0 for 2.5 and below

is_open = open_df.values #1 for open 0 for closed


##########################################################################
##########Data gathered, lets make observed & expected dataframes#########
##########################################################################

dfobserved =  pd.DataFrame(columns = ['Business Open' , 'Business Closed', 'Total Stars'], index=['Stars above 2.5', 'Stars below 2.5', 'Total Operating'])

##4 variables for each subcat
open_above = 0
open_below = 0
closed_above = 0
closed_below = 0

for idx, bus_status in enumerate(is_open):

    star_status = bin_stars[idx]

    if bus_status == 1:
        if star_status == 1:
            open_above += 1
        else:
            open_below += 1
    else:
        if star_status == 1:
            closed_above += 1
        else:
            closed_below += 1




In [3]:
#####lets fill in observed df########
dfobserved["Business Open"]["Stars above 2.5"]=open_above

#TODO
conclusions = 'your conclusions'
print('Conclusions:\t', conclusions)

Conclusions:	 your conclusions


## Q2. Correlation coefficients

In [4]:
#TODO pearson coefficient


In [5]:
#TODO spearman coefficient


In [6]:
#TODO
conclusions = 'your conclusions'
print('Conclusions:\t', conclusions)

Conclusions:	 your conclusions


## Q3. Association rule

In [7]:
import re
df_AZ = dfBus[dfBus["state"]=="AZ"]
raw_categories = df_AZ.loc[:,'categories']

cats = []
for cat in raw_categories:
    cats.append(cat)


#TODO


In [13]:

nested = []

for desc in cats:
    sub = []
    key_words = re.findall('[a-zA-Z]+', desc)
    for word in key_words:
        sub.append(word)
    nested.append(sub)



In [9]:
#TODO
conclusions = 'your conclusions'
print('Conclusions:\t', conclusions)

Conclusions:	 your conclusions


## Q4. Power law distribution

In [10]:
#TODO ccdf


In [11]:
#TODO powerlaw


In [12]:
#TODO
conclusions = 'your conclusions'
print('Conclusions:\t', conclusions)

Conclusions:	 your conclusions
